<a href="https://colab.research.google.com/github/AbhinavKumar07/P115/blob/main/Project_C115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the dataset from Github repository
!git clone https://github.com/procodingclass/product_dataset.git

Cloning into 'product_dataset'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 3.08 MiB | 6.94 MiB/s, done.


In [ ]:
# Make a dataframe using the 'pandas' module
import pandas as pd
dataframe = pd.read_excel('/content/product_dataset/updated_product_dataset.xlsx')
print(dataframe.head())

    Emotion                                               Text
0  Positive  close approximation red octane mat bought one ...
1   Neutral  little lumpy mat great foam padding itâ€™s use...
2  Positive  great pad love ddr not want metal pad get work...
3  Positive  excellent pad great product highly responsive ...
4  Positive  awesome great ddr pad works perfectly pc stepm...


In [ ]:
# Get unique emotions from the 'Emotion' column in the Dataset
dataframe["Emotion"].unique()

array(['Positive', 'Neutral', 'Negative'], dtype=object)

In [ ]:
# Add Labels to the dataset
encode_emotions = {"Neutral": 0, "Positive": 1, "Negative": 2}

In [ ]:
# replace the emotions with the Label
dataframe.replace(encode_emotions, inplace = True)
dataframe.head(100)

,Emotion,Text
0,1,close approximation red octane mat bought one ...
1,0,little lumpy mat great foam padding itâ€™s use...
2,1,great pad love ddr not want metal pad get work...
3,1,excellent pad great product highly responsive ...
4,1,awesome great ddr pad works perfectly pc stepm...
...,...,...
95,1,amazing game fantastic game buy thatâ€™s all say
96,1,ignore fake 1star reviews game really goodit u...
97,1,hours fun graphics compellingi great time try...
98,1,last us thanks love


In [ ]:
# Convert the dataframe into list for easy processing
training_sentences = []
training_labels = []

for i in range(len(dataframe)):
  sentence = dataframe.loc[i, "Text"]
  training_sentences.append(sentence)
  label = dataframe.loc[i, "Emotion"]
  training_labels.append(label)

# printing 10th element
training_sentences[10], training_labels[10]


('arrived early included blank case wont able test game get switch tried brothers device recognized card smash digital download already arrived ahead schedule well packed even blank case wasnt expectingread full review',
 1)

In [ ]:
# Tokenize and pad your data
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
# Converting to numpy array
import numpy as np

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)

In [ ]:
from keras.api._v2.keras import activations
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  LSTM , Embedding , Dense , Dropout , Conv1D , MaxPooling1D


# Create and compile model

model = tf.keras.Sequential([
    Embedding(vocab,embedding_dim,input_length = max_length),
    Dropout(0.2),

    Conv1D(filters = 64 , kernel_size = 3 , activation = 'relu'),
    MaxPooling1D(pool_size = 3),
    Conv1D(filters = 32 , kernel_size = 3 , activation = 'relu'),
    MaxPooling1D(pool_size = 3),
    LSTM(32),

    Dense(32 , activation = 'relu'),
    Dropout(0.2),
    Dense(16 , activation = 'relu'),
    Dense(8 , activation = 'softmax')
])
model.compile(loss = 'sparse_categorical_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 dropout (Dropout)           (None, 100, 16)           0         
                                                                 
 conv1d (Conv1D)             (None, 98, 64)            3136      
                                                                 
 max_pooling1d (MaxPooling1  (None, 32, 64)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 30, 32)            6176      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 10, 32)            0         
 g1D)                                                   

In [ ]:
# Train Model
model_history = model.fit(training_padded , training_labels , epochs = 30 , verbose = 1)

Epoch 1/30
314/671 [=============>................] - ETA: 6s - loss: 0.5222 - accuracy: 0.9150

In [ ]:
#Save and run model
model.save('reviews.h5')

review = ["close approximation red octane mat bought one liked bought another onethis ddr mat 1 thick foam pad much comfortable stock mats put yoga mats even padding easier stay aligned mat dancing theres slight dropoff edges four main pads feel feetit comes two bags one containing shell folded one containing pieces foam interlock like puzzle pieces sure assemble correctly form perfect square flip pieces upside necessary surface takes flatten put together eventuallyi red octane mat seems bit better theyre quite similar button isnt very responsive works isnt very important cant speak durabilitythe instructions dont mention wii compatibility works fine wiiread full review"]
sequences = tokenizer.texts_to_sequences(review)
result = model.predict(model)

class_prediction = np.argmax(result, axis = 1)
class_prediction